In [8]:
import pandas as pd
import numpy as np
import math
from typing_extensions import final
from tabulate import tabulate
from collections import defaultdict

In [9]:
df = pd.read_csv("Decision Grid.csv", usecols=[1, 2, 3, 4, 5, 6, 7, 8])

df.columns = ['Treated', 'DLT', 'Pending', 'State', 'Orig_33_Action', 'IQ_33_Action', 'Orig_6_Action', 'IQ_6_Action']

decision_dict_IQ_33 = df.set_index(['Treated', 'DLT', 'Pending', 'State'])['IQ_33_Action'].to_dict()
decision_dict_33 = df.set_index(['Treated', 'DLT', 'Pending', 'State'])['Orig_33_Action'].to_dict()
decision_dict_IQ_6 = df.set_index(['Treated', 'DLT', 'Pending', 'State'])['IQ_6_Action'].to_dict()
decision_dict_6 = df.set_index(['Treated', 'DLT', 'Pending', 'State'])['Orig_6_Action'].to_dict()

In [10]:
# visualization of the processed data
print(df.head(15))

    Treated  DLT  Pending   State Orig_33_Action IQ_33_Action Orig_6_Action  \
0         0    0        0    open           Stay         Stay          Stay   
1         0    0        0  closed           Stay         Stay          Stay   
2         1    0        0    open           Stay         Stay          Stay   
3         1    0        0  closed           Stay         Stay          Stay   
4         1    1        0    open           Stay         Stay          Stay   
5         1    1        0  closed           Stay         Stay          Stay   
6         1    0        1    open           Stay         Stay          Stay   
7         1    0        1  closed           Stay         Stay          Stay   
8         2    0        0    open           Stay         Stay          Stay   
9         2    0        0  closed           Stay         Stay          Stay   
10        2    1        0    open           Stay         Stay          Stay   
11        2    1        0  closed           Stay    

In [11]:
print(f"Decision for Row index 29 3+3 (3,0,1,open): {decision_dict_33.get((3, 0, 1, 'open'))}")
print(f"Decision for Row index 29 IQ 3+3 (3,0,1,open): {decision_dict_IQ_33.get((3, 0, 1, 'open'))}")
print(f"Decision for Row index 29 rolling 6 (3,0,1,open): {decision_dict_IQ_6.get((3, 0, 1, 'open'))}")
print(f"Decision for Row index 29 IQ rolling 6 (3,0,1,open): {decision_dict_6.get((3, 0, 1, 'open'))}")


Decision for Row index 29 3+3 (3,0,1,open): Suspend
Decision for Row index 29 IQ 3+3 (3,0,1,open): Stay
Decision for Row index 29 rolling 6 (3,0,1,open): Stay
Decision for Row index 29 IQ rolling 6 (3,0,1,open): Stay


Data is processed into four python dictionaries: one for each of the methodologies for which the simulation will run (original 3+3, IQ 3+3, original rolling 6, IQ rolling 6). The values in each of the columns of these methodologies are as follows.

 Defines what dose level the next patient should be treated at, based on the current number of patients being treated, the number of reported DLTs, and the number of results pending at the current dose level.

Possible Dispositions:

1. Up          = start testing patients at the next higher dose level
2. Down     = start testing patients at the next lower dose level
3. Stay       = keep testing patients at the current dose level
4. Suspend = wait for additional testing to finish at current dose
                      level, before starting a test on another patient.
5. MTD       = current dose is the Maximum Tolerated Dose (MTD)


The value of having this as a python dictionary is that all values can be looked up in place for a much faster run time. So if we want to know what to do for IQ 3+3 when we have Treated = 3, DLT = 0, Pending = 1, and State = open, we can just query the corresponding dictionary with "decision_dict_IQ_33.get((3, 0, 1, 'open'))" to get "Stay"

All that is left is to simulate the actions of the dispositions while also considering possible dropout of people that develop DLT.




In [12]:
Starting_Dose_Level = 2
Lowest_Possible_Dose_Level = 1
Highest_Possible_Dose_Level = 5

#(maximum duration interested patients will wait for a slot)
Waitlist_Time = 0
#(duration of DLT evaluation period)
Course_Length = 28
#(percent who will fail the screening process)
Screen_Failure_Prob = 0.30
#(percent inevaluable.  Actual inevaluable rate will be lower due to competing DLT risk)
Inevaluable_Prob = 0.20


# DLT Probability Function (defines the probability of a patient having a DLT event, based on dose)
"""
def get_dlt_probability(CurDoseLevel):

    prob_of_dlt = (
0.10*(CurDoseLevel-2)*(CurDoseLevel-3)*(CurDoseLevel-4)*(CurDoseLevel-5)*(CurDoseLevel-6)/
      ((1-2)*(1-3)*(1-4)*(1-5)*(1-6)) +

0.15*(CurDoseLevel-1)*(CurDoseLevel-3)*(CurDoseLevel-4)*(CurDoseLevel-5)*(CurDoseLevel-6)/
      ((2-1)*(2-3)*(2-4)*(2-5)*(2-6)) +

0.25*(CurDoseLevel-1)*(CurDoseLevel-2)*(CurDoseLevel-4)*(CurDoseLevel-5)*(CurDoseLevel-6)/
      ((3-1)*(3-2)*(3-4)*(3-5)*(3-6)) +

0.30*(CurDoseLevel-1)*(CurDoseLevel-2)*(CurDoseLevel-3)*(CurDoseLevel-5)*(CurDoseLevel-6)/
      ((4-1)*(4-2)*(4-3)*(4-5)*(4-6)) +

0.35*(CurDoseLevel-1)*(CurDoseLevel-2)*(CurDoseLevel-3)*(CurDoseLevel-4)*(CurDoseLevel-6)/
      ((5-1)*(5-2)*(5-3)*(5-4)*(5-6)) +

0.50*(CurDoseLevel-1)*(CurDoseLevel-2)*(CurDoseLevel-3)*(CurDoseLevel-4)*(CurDoseLevel-5)/
      ((6-1)*(6-2)*(6-3)*(6-4)*(6-5))
  )

    return prob_of_dlt


"""

def get_dlt_probability(dose_level):
    """Using values from the paper"""
    # Formula: 100 * (0.5 + atan(0.2 * pi * (curr_dose_level - 8.5)) / pi)
    val = 0.5 + (math.atan(0.2 * math.pi * (dose_level - 8.5)) / math.pi)
    return max(0.0, min(1.0, val))


#Patient Inter-Arrival Time (days between arrivals)
def sample_interarrival():
    return math.floor(np.random.exponential(10))  # Mean of 10 days

#Screening Duration Distribution (length of screening process)
def sample_screening_duration():
    # beta(0, 28, 1, 1) is a uniform distribution between 0 and 28
    return math.floor(np.random.uniform(0, 28))

#Time Until DLT Event (distribution of times before the patient will have the DLT event)
def sample_dlt_timing():
    # beta(0, CourseLength, 1.5, 1) skewed toward the end of the cycle
    return math.floor(np.random.beta(1.5, 1) * Course_Length)

#Time Until Inevaluable (distribution of times before the patient will become inevaluable)
def sample_inevaluable_timing():
    # beta(0,CourseLength,1,1) is a Uniform(0, 28) distribution
    return math.floor(np.random.beta(1, 1) * Course_Length)


Now to make a class for the average patient

In [13]:
class Patient:
  def __init__(self, arrival_time, dose, ID, course_length = Course_Length):
    self.ID = ID
    self.arrival_time = arrival_time
    self.dose = dose #int 1-5 inclusive
    self.course_length = course_length #length of study
    self.started_treatment = False
    self.treatment_start_time = None

    self.screening_outcome, self.days_to_screening_outcome = self.determine_screening_outcome()

    self.started_screening = False
    self.screening_start_time = None
    self.screening_end_time = None

    self.treatment_outcome, self.days_to_treatment_outcome = self.determine_patient_outcome()

    self.started_treatment = False
    self.treatment_start_time = None
    self.resolution_time = None


  def determine_screening_outcome(self):
    screen_duration = sample_screening_duration()
    if np.random.random() < Screen_Failure_Prob:
      return "fail", screen_duration
    else:
      return "pass", screen_duration

  def start_screening(self, current_clock_time):
    self.started_screening = True
    self.screening_start_time = current_clock_time
    self.screening_end_time = self.screening_start_time + self.days_to_screening_outcome


  # returns "dlt", "ineval", or "pass", as well as a duration til it happens
  def determine_patient_outcome(self):
    dlt_prob = get_dlt_probability(self.dose)
    ineval_prob = Inevaluable_Prob

    time_to_ineval = -1
    time_to_dlt = -1

    if np.random.random() < ineval_prob:
      time_to_ineval = sample_inevaluable_timing()

    if np.random.random() < dlt_prob:
      time_to_dlt = sample_dlt_timing()

    # if these both happen, we choose the one that will happen first in the timeline
    if time_to_ineval > -1 and time_to_dlt > -1:
      if time_to_ineval < time_to_dlt:
        return "ineval", time_to_ineval
      else:
        return "dlt", time_to_dlt
    elif time_to_ineval > -1:
      return "ineval", time_to_ineval
    elif time_to_dlt > -1:
      return "dlt", time_to_dlt
    else:
      return "pass", self.course_length

  # current_clock_time will be an int representation of days since trial
  # resolution_time is the day they pass, get dlt, or become ineval
  def start_treatment(self, current_clock_time):
    self.started_treatment = True
    self.treatment_start_time = current_clock_time
    self.resolution_time = self.treatment_start_time + self.days_to_treatment_outcome


In [14]:
def archive(treated_list, pending_list):
  treated_list.extend(pending_list)
  pending_list.clear()


In [15]:

def simulation(trial_type):
  if trial_type == 1:
    decision_dict = decision_dict_33
  elif trial_type == 2:
    decision_dict = decision_dict_IQ_33
  elif trial_type == 3:
    decision_dict = decision_dict_6
  elif trial_type == 4:
    decision_dict = decision_dict_IQ_6
  else:
    print("Invalid trial type: Please choose a value 1-4")
    return None, None, None, None

  # debug error var
  end_on_error = False

  trial_status = True
  # options are Stay, Up, Down, Suspend, MTD
  current_trial_event = "Stay"
  dose_level = Starting_Dose_Level
  max_dose_level = Highest_Possible_Dose_Level
  min_dose_level = Lowest_Possible_Dose_Level
  final_dose_level = None

  # this will be how many patients are not taken because of lack of a queue
  overlooked_patients = 0
  # starts on day 1 of trial
  current_clock_time = 1
  # always need one at the start
  patient_needed = True
  days_before_next_patient = sample_interarrival()

  patientID = 1

  screening_list = []
  pending_list = []
  # (list of patients all patients: treated, DLT, and pending)
  treated_list = []
  total_inevals = 0
  if dose_level == max_dose_level:
    patient_states = {"treated": 0, "DLT": 0, "pending": 0, "AboveLevelState": "closed"}
  else:
    patient_states = {"treated": 0, "DLT": 0, "pending": 0, "AboveLevelState": "open"}

  # limited to detect infinite loops
  while trial_status:
    """
    check for new patients and add them to the screening list if they are desired
    check the screening list patients and add them to the patient list if they are ready
    check the patients in the pending list to see if they have passed or otherwise and act accordingly
    check for trial status change
      do stuff if changed
    """

    # checking for new patients, add them to screening
    # without queue, turned away if not needed that day
    # Patient cannot start screening unless they are needed, lest we waste their time
    if days_before_next_patient <= 0 and patient_needed:
      days_before_next_patient = sample_interarrival()
      new_patient = Patient(current_clock_time, dose_level, patientID)
      patientID += 1
      new_patient.start_screening(current_clock_time)
      screening_list.append(new_patient)
      # uf they are screening, they take up a spot in the study
      patient_states["pending"] += 1
      patient_states["treated"] += 1
    elif days_before_next_patient == 0:
      days_before_next_patient = sample_interarrival()
      overlooked_patients += 1
    else:
      days_before_next_patient -= 1


    # checking screening list (patient queued if not needed, no need to waste a screened patient, dose level subject to change if Up or Down)
    # below, switch to the commented block to slightly decrease study time, doesn't match study metrics
    """
    while screening_list and screening_list[0].screening_end_time <= current_clock_time:
      patient = screening_list[0]
      if patient.screening_outcome == "fail":
        screening_list.pop(0)
        patient_states["pending"] -= 1
        patient_states["treated"] -= 1
        # Move to the next person in the list for the same day
        continue
      current_trial_event = decision_dict.get((patient_states['treated'], patient_states['DLT'], patient_states['pending'], patient_states['AboveLevelState']), "Error")

      if patient.screening_outcome == "pass" and current_trial_event != "Suspend":
        screening_list.pop(0)
        if patient.dose != dose_level:
          patient.dose = dose_level
        patient.start_treatment(current_clock_time)
        pending_list.append(patient)
      else:
        break
    """

    if screening_list and screening_list[0].screening_end_time <= current_clock_time:
      patient = screening_list.pop(0)
      if patient.screening_outcome == "fail":
        patient_states["pending"] -= 1
        patient_states["treated"] -= 1
      else:
        if patient.dose != dose_level:
          patient.dose = dose_level
        patient.start_treatment(current_clock_time)
        pending_list.append(patient)



    # checking pending list
    for patient in pending_list[:]:
      # will never trigger twice cause clock time changes. check will happen every time though, could optomize to event checking
      # no mention of "pass" because num treated alredy reflects that patient
      if patient.resolution_time == current_clock_time:
        pending_list.remove(patient)
        patient_states["pending"] -= 1
        # inevals not archived, just noted as gone
        # decrement because they do leave the study
        if patient.treatment_outcome == "ineval":
          patient_states["treated"] -= 1
          total_inevals += 1
        elif patient.treatment_outcome == "dlt":
          patient_states["DLT"] += 1
        treated_list.append(patient)

    # options are Stay, Up, Down, Suspend, MTD
    current_trial_event = decision_dict.get((patient_states['treated'], patient_states['DLT'], patient_states['pending'], patient_states['AboveLevelState']), "Error")

    if current_trial_event == "Up":
      max_completed_dose_level = dose_level
      dose_level += 1
      patient_needed = True

      archive(treated_list, pending_list)

      new_state = "closed" if dose_level == max_dose_level else "open"
      # this represents a case where there is someone(s) is still screening and we will adjust the dose for them (only happens for IQ)
      if screening_list:
        still_screening = len(screening_list)
        patient_states.update({"treated": still_screening, "DLT": 0, "pending": still_screening, "AboveLevelState": new_state})
      else:
        patient_states.update({"treated": 0, "DLT": 0, "pending": 0, "AboveLevelState": new_state})

    elif current_trial_event == "Down":
      if dose_level == min_dose_level:
        final_dose_level = dose_level -1
        trial_status = False
        break

      dose_level -= 1
      patient_needed = True

      archive(treated_list, pending_list)

      # this represents an uncommon case where there are 2 dlts and the dose goes down, but someone(s) is still screening and we adjust the dose for them
      if screening_list:
        still_screening = len(screening_list)
        patient_states.update({"treated": still_screening, "DLT": 0, "pending": still_screening, "AboveLevelState": "closed"})
      else:
        patient_states.update({"treated": 0, "DLT": 0, "pending": 0, "AboveLevelState": "closed"})


    current_trial_event = decision_dict.get((patient_states['treated'], patient_states['DLT'], patient_states['pending'], patient_states['AboveLevelState']), "Error")

    if current_trial_event == "Stay":
      patient_needed = True
      # cohort expansion logic for when extra cohort needed to determine MTD
      # if trial_type == 1 and batch_counter == 3 and patient_states['pending'] == 0 and screening_list == []:
      #   batch_counter = 0
    elif current_trial_event == "Suspend":
      patient_needed = False

    elif current_trial_event == "MTD":
      archive(treated_list, pending_list)

      final_dose_level = dose_level
      trial_status = False

    # haven't triggered this block in a long time, yay
    if current_trial_event == "Error":
      archive(treated_list, pending_list)
      final_dose_level = dose_level

      if dose_level == min_dose_level:
        result_summary = "Terminated: Too Toxic at Lowest Level"
      else:
        result_summary = "Terminated: Error/Inconclusive"

      print("Error in chart reached")
      print(f"Reason: {result_summary}")
      end_on_error = True
      trial_status = False

    current_clock_time += 1


  if final_dose_level is None:
    print("trial timeout")
    final_dose_level = dose_level

  # returns list of all treated/dlt patients, days_taken, final_dose_level, total_inevals
  return treated_list, current_clock_time, final_dose_level, total_inevals


In [16]:
def debug_log(log, day, event, action, states):
  return f"Day: {day:<4} | Event: {event:<10} | {action:<65} | States: {states} \n"

# debug for simulations
def simulation_with_debug(trial_type):
  event_log = ""

  if trial_type == 1:
    decision_dict = decision_dict_33
    event_log += "Beginning 3+3 Simulation\n"
  elif trial_type == 2:
    decision_dict = decision_dict_IQ_33
    event_log += "Beginning IQ 3+3 Simulation\n"
  elif trial_type == 3:
    decision_dict = decision_dict_6
    event_log += "Beginning Rolling 6 Simulation\n"
  elif trial_type == 4:
    decision_dict = decision_dict_IQ_6
    event_log += "Beginning IQ Rolling 6 Simulation\n"
  else:
    print("Invalid trial type: Please choose a value 1-4")
    return None, None, None, None, None, None

  # debug error var
  end_on_error = False

  trial_status = True
  # options are Stay, Up, Down, Suspend, MTD
  current_trial_event = "Stay"
  dose_level = Starting_Dose_Level
  max_dose_level = Highest_Possible_Dose_Level
  min_dose_level = Lowest_Possible_Dose_Level
  final_dose_level = None

  # this will be how many patients are not taken because of lack of a queue
  overlooked_patients = 0
  # starts on day 1 of trial
  current_clock_time = 1
  # always need one at the start
  patient_needed = True
  days_before_next_patient = sample_interarrival()

  patientID = 1

  screening_list = []
  pending_list = []
  # (list of patients all patients: treated, DLT, and pending)
  treated_list = []
  total_inevals = 0
  if dose_level == max_dose_level:
    patient_states = {"treated": 0, "DLT": 0, "pending": 0, "AboveLevelState": "closed"}
  else:
    patient_states = {"treated": 0, "DLT": 0, "pending": 0, "AboveLevelState": "open"}

  # log (for debug: not needed for many simulations in a row, wasted compute)
  state_log = []

  # add and current_clock_time < 4000 to avoid infinite loop,
  # however it holds back testing cases like super high ineval rates or arrival/trial times
  while trial_status:
    """
    check for new patients and add them to the screening list if they are desired
    check the screening list patients and add them to the patient list if they are ready
    check the patients in the pending list to see if they have passed or otherwise and act accordingly
    check for trial status change
      do stuff if changed
    """

    # checking for new patients, add them to screening
    # without queue, turned away if not needed that day
    if days_before_next_patient <= 0 and patient_needed:
      days_before_next_patient = sample_interarrival()
      new_patient = Patient(current_clock_time, dose_level, patientID)
      patientID += 1
      new_patient.start_screening(current_clock_time)
      screening_list.append(new_patient)
      # if they are screening, they take up a spot in the study
      patient_states["pending"] += 1
      patient_states["treated"] += 1
      event_log += debug_log(event_log, current_clock_time, current_trial_event, f"Adding new patient {new_patient.ID} to screening.", patient_states)
    elif days_before_next_patient == 0:
      days_before_next_patient = sample_interarrival()
      overlooked_patients += 1
    else:
      days_before_next_patient -= 1


    # checking screening list (patient queued if not needed, no need to waste a screened patient, dose level subject to change if Up or Down)
    # no need for a loop if we just want to add 1 at most on a given day
    """
    while screening_list and screening_list[0].screening_end_time <= current_clock_time:
      patient = screening_list[0]
      if patient.screening_outcome == "fail":
        screening_list.pop(0)
        patient_states["pending"] -= 1
        patient_states["treated"] -= 1
        event_log += debug_log(event_log, current_clock_time, current_trial_event, f"Patient {patient.ID} failed screening.", patient_states)
        continue

      current_trial_event = decision_dict.get((patient_states['treated'], patient_states['DLT'], patient_states['pending'], patient_states['AboveLevelState']), "Error")

      if patient.screening_outcome == "pass" and current_trial_event != "Suspend":
        screening_list.pop(0)
        if patient.dose != dose_level:
          patient.dose = dose_level
        patient.start_treatment(current_clock_time)
        pending_list.append(patient)
        event_log += debug_log(event_log, current_clock_time, current_trial_event, f"Patient {patient.ID} starts treatment, at dose level {patient.dose}.", patient_states)
      else:
        break
    """
    if screening_list and screening_list[0].screening_end_time <= current_clock_time:
      patient = screening_list.pop(0)
      if patient.screening_outcome == "fail":
        patient_states["pending"] -= 1
        patient_states["treated"] -= 1
        event_log += debug_log(event_log, current_clock_time, current_trial_event, f"Patient {patient.ID} failed screening.", patient_states)
      # even if suspend, these patients are part of pending and need to resolve
      else:
        if patient.dose != dose_level:
          patient.dose = dose_level
        patient.start_treatment(current_clock_time)
        pending_list.append(patient)
        event_log += debug_log(event_log, current_clock_time, current_trial_event, f"Patient {patient.ID} starts treatment, at dose level {patient.dose}.", patient_states)

    # checking pending list
    for patient in pending_list[:]:
      # will never trigger twice cause clock time changes. check will happen every time though, could optomize to event checking
      # no mention of "pass" because num treated alredy reflects that patient
      if patient.resolution_time == current_clock_time:
        pending_list.remove(patient)
        patient_states["pending"] -= 1
        if patient.treatment_outcome == "pass":
          event_log += debug_log(event_log, current_clock_time, current_trial_event, f"Patient {patient.ID} passes treatment.", patient_states)
        # inevals not archived, just noted as gone
        # decrement because they do leave the study
        if patient.treatment_outcome == "ineval":
          patient_states["treated"] -= 1
          total_inevals += 1
          event_log += debug_log(event_log, current_clock_time, current_trial_event, f"Removing patient {patient.ID} for ineval.", patient_states)
        elif patient.treatment_outcome == "dlt":
          patient_states["DLT"] += 1
          event_log += debug_log(event_log, current_clock_time, current_trial_event, f"Patient {patient.ID} develops DLT.", patient_states)
        treated_list.append(patient)

    # options are Stay, Up, Down, Suspend, MTD
    current_trial_event = decision_dict.get((patient_states['treated'], patient_states['DLT'], patient_states['pending'], patient_states['AboveLevelState']), "Error")


    if current_trial_event == "Up":
      max_completed_dose_level = dose_level
      dose_level += 1
      patient_needed = True

      ids = [p.ID for p in pending_list]
      event_log += debug_log(event_log, current_clock_time, current_trial_event, f"Moving patients {ids} to treated from pending, dose goes up.", patient_states)

      archive(treated_list, pending_list)

      new_state = "closed" if dose_level == max_dose_level else "open"
      # this represents a case where there is someone(s) is still screening and we will adjust the dose for them (only happens for IQ)
      if screening_list:
        still_screening = len(screening_list)
        patient_states.update({"treated": still_screening, "DLT": 0, "pending": still_screening, "AboveLevelState": new_state})
      else:
        patient_states.update({"treated": 0, "DLT": 0, "pending": 0, "AboveLevelState": new_state})


    # this dose level is now considered toxic
    elif current_trial_event == "Down":
      if dose_level == min_dose_level:
        final_dose_level = dose_level -1
        trial_status = False
        break

      dose_level -= 1
      patient_needed = True

      ids = [p.ID for p in pending_list]
      event_log += debug_log(event_log, current_clock_time, current_trial_event, f"Moving patients {ids} to treated from pending, dose goes down.", patient_states)

      archive(treated_list, pending_list)

      # this represents an uncommon case where there are 2 dlts and the dose goes down, but someone(s) is still screening and we adjust the dose for them
      if screening_list:
        still_screening = len(screening_list)
        patient_states.update({"treated": still_screening, "DLT": 0, "pending": still_screening, "AboveLevelState": "closed"})
      else:
        patient_states.update({"treated": 0, "DLT": 0, "pending": 0, "AboveLevelState": "closed"})

    elif current_trial_event == "MTD":
      ids = [p.ID for p in pending_list]
      event_log += debug_log(event_log, current_clock_time, current_trial_event, f"Moving patients {ids} to treated from pending, dose is MTD.", patient_states)

      archive(treated_list, pending_list)

      final_dose_level = dose_level
      trial_status = False

      # print(f"MTD Reached on day {current_clock_time}, with dose level {final_dose_level}")

    # current trial state may have changed in the up block if 3 patients were all in screening and now instead of stay like you would in an up, it goes to suspend
    current_trial_event = decision_dict.get((patient_states['treated'], patient_states['DLT'], patient_states['pending'], patient_states['AboveLevelState']), "Error")

    if current_trial_event == "Stay":
      patient_needed = True
      #  if extra cohort needed for 33 mtd
      # if trial_type == 1 and batch_counter == 3 and patient_states['pending'] == 0 and screening_list == []:
      #   event_log += debug_log(event_log, current_clock_time, current_trial_event, f"Expanding cohort to 6 at dose level: {dose_level}", patient_states)
    elif current_trial_event == "Suspend":
      patient_needed = False

    if current_trial_event == "Error":
      ids = [p.ID for p in pending_list]
      event_log += debug_log(event_log, current_clock_time, current_trial_event, f"Moving patients {ids} to treated from pending, error.", patient_states)

      archive(treated_list, pending_list)
      final_dose_level = dose_level

      if dose_level == min_dose_level:
        result_summary = "Terminated: Too Toxic at Lowest Level"
      else:
        result_summary = "Terminated: Error/Inconclusive"

      print("Error in chart reached")
      print(f"Reason: {result_summary}")
      end_on_error = True
      trial_status = False

    current_trial_event = decision_dict.get((patient_states['treated'], patient_states['DLT'], patient_states['pending'], patient_states['AboveLevelState']), "Error")


    snapshot = patient_states.copy()
    snapshot['day'] = current_clock_time
    snapshot['dose'] = dose_level
    snapshot['action'] = current_trial_event
    state_log.append(snapshot)

    current_clock_time += 1


  event_log += f"Final Dose Level: {final_dose_level}\n"
  event_log += f"Total Time Taken: {current_clock_time} days or ~{round(current_clock_time / 30.375 , 2)} months\n"
  # print(f"Overlooked patients: {overlooked_patients}")

  # return state_log if you want a break down of the states day by day
  # returns list of all treated/dlt patients, days_taken, final_dose_level, total_inevals
  return treated_list, current_clock_time, final_dose_level, total_inevals, state_log, event_log, end_on_error


In [17]:
def get_metrics(simulation_type, num_repetitions):
  metrics, dose_stats = [], defaultdict(lambda: {"dlts": 0, "total": 0})
  for i in range(num_repetitions):
    # treated_list now includes ALL patients (total treated)
    treated_list, days_taken, final_dose, total_inevals = simulation(simulation_type)

    # Identify Evaluable patients (outcome is not 'ineval')
    evaluable_list = [p for p in treated_list if p.treatment_outcome != "ineval"]
    num_treated = len(treated_list)
    num_evaluable = len(evaluable_list)

    num_DLT_above_MTD = 0
    for p in treated_list:
      dose_stats[p.dose]["total"] += 1
      if p.treatment_outcome == "dlt":
        dose_stats[p.dose]["dlts"] += 1
        if p.dose > final_dose: num_DLT_above_MTD += 1

    metrics.append({
      "days_taken": days_taken,
      "final_dose": final_dose,
      "num_treated": num_treated,      # Total count
      "num_evaluable": num_evaluable,  # Completed DLT window
      "num_DLT_above_MTD": num_DLT_above_MTD
    })
  return pd.DataFrame(metrics), dose_stats

def get_dlt_probability(dose_level):
    """Using the arctan formula from the paper to get ground truth toxicity."""
    if dose_level is None or dose_level <= 0:
        return None
    val = 0.5 + (math.atan(0.2 * math.pi * (dose_level - 8.5)) / math.pi)
    return max(0.0, min(1.0, val))

def format_theoretical_dlt_col(max_level=5):
    """Formats the 'Ground Truth' column based on the mathematical model."""
    lines = ["NA  NA"]
    for level in range(1, max_level + 1):
        prob = get_dlt_probability(level)
        rate = prob * 100
        lines.append(f"{str(level).ljust(2)}  {rate:.1f}%")
    return "\n".join(lines)


def bucket_dose(val): return int(val) if val and val > 0 else "NA"

def format_pct(val):
  if val == 0: return "0.0%"
  return "<1.0%" if 0 < val < 1.0 else f"{val:.1f}%"

def get_mtd_breakdown(doses, max_lvl=5):
  bucketed = doses.apply(bucket_dose)
  probs = (bucketed.value_counts() / len(doses) * 100)
  order = ["NA"] + list(range(1, max_lvl + 1))
  return probs.reindex(order, fill_value=0.0).apply(format_pct).to_frame("% MTD At level")

def format_stat_str(ser):
  m, med = round(ser.mean(), 1), round(ser.median(), 1)
  mi, ma = round(ser.min(), 1), round(ser.max(), 1)
  f_mi = int(mi) if mi == int(mi) else mi
  f_ma = int(ma) if ma == int(ma) else ma
  return f"{m}, {med}\n({f_mi}-{f_ma})"

def format_mtd_col(doses, max_lvl=5):
  df = get_mtd_breakdown(doses, max_lvl)
  return "\n".join([f"{str(lvl).ljust(2)}  {row['% MTD At level']}" for lvl, row in df.iterrows()])

In [18]:
def run_batch_report(scenarios, sim_types, num_reps):
  global Starting_Dose_Level, Highest_Possible_Dose_Level, Lowest_Possible_Dose_Level
  global Waitlist_Time, Course_Length, Screen_Failure_Prob, Inevaluable_Prob

  type_map = {1: "3+3", 2: "IQ 3+3", 3: "Rolling 6", 4: "IQ Rolling 6"}
  table_rows, headers = [], []

  for p in scenarios:
    # Update global scenario parameters
    Starting_Dose_Level = p.get("Starting_Dose_Level", Starting_Dose_Level)
    Highest_Possible_Dose_Level = p.get("Highest_Possible_Dose_Level", Highest_Possible_Dose_Level)
    Lowest_Possible_Dose_Level = p.get("Lowest_Possible_Dose_Level", Lowest_Possible_Dose_Level)
    Waitlist_Time = p.get("Waitlist_Time", Waitlist_Time)
    Course_Length = p.get("Course_Length", Course_Length)
    Screen_Failure_Prob = p.get("Screen_Failure_Prob", Screen_Failure_Prob)
    Inevaluable_Prob = p.get("Inevaluable_Prob", Inevaluable_Prob)

    row = {"Variation": p.get("name", "Unnamed")}
    tr_cols, ev_cols, mo_cols, mtd_cols, dlt_sum = {}, {}, {}, {}, []

    for st in sim_types:
      # Run simulation and get metrics
      df, _ = get_metrics(st, num_reps) # stats (_) no longer needed for model column
      df['months_taken'] = df['days_taken'] / 30.4375
      lbl = type_map.get(st, f"T{st}")

      # Collect columns for this simulation type
      tr_cols[f"#Treated\n{lbl}\nMean, Med\n(range)"] = format_stat_str(df['num_treated'])
      ev_cols[f"#Evaluable\n{lbl}\nMean, Med\n(range)"] = format_stat_str(df['num_evaluable'])
      mo_cols[f"#Months\n{lbl}\nMean, Med\n(range)"] = format_stat_str(df['months_taken'])

      dlt_sum.append(f"{lbl}: {df['num_DLT_above_MTD'].mean():.2f}")
      mtd_cols[f"{lbl}\n% MTD\nAt level"] = format_mtd_col(df['final_dose'].copy(), Highest_Possible_Dose_Level)

    # Build the horizontal row in exact eTable 3 order
    row.update(tr_cols)
    row.update(ev_cols)
    row.update(mo_cols)
    row["Mean\n#DLTs\nabove\nMTD"] = "\n".join(dlt_sum)
    row.update(mtd_cols)

    # New: Use the theoretical formula for the final column
    row["DLT rate\n(based\non model)"] = format_theoretical_dlt_col(Highest_Possible_Dose_Level)

    if not headers:
        headers = list(row.keys())
    table_rows.append(list(row.values()))

  print(tabulate(table_rows, headers=headers, tablefmt="grid"))

In [19]:
def run_debug_test(sim_type=1, sdl=2, hpd=5, lpd=1, wt=0, cl=28, sfp=0.3, ip=0.2):
  global Starting_Dose_Level, Highest_Possible_Dose_Level, Lowest_Possible_Dose_Level
  global Waitlist_Time, Course_Length, Screen_Failure_Prob, Inevaluable_Prob

  orig = (Starting_Dose_Level, Highest_Possible_Dose_Level, Lowest_Possible_Dose_Level,
          Waitlist_Time, Course_Length, Screen_Failure_Prob, Inevaluable_Prob)

  Starting_Dose_Level, Highest_Possible_Dose_Level, Lowest_Possible_Dose_Level = sdl, hpd, lpd
  Waitlist_Time, Course_Length, Screen_Failure_Prob, Inevaluable_Prob = wt, cl, sfp, ip

  # res = simulation_with_debug(sim_type)
  # print(res[5]) # Prints the event_log

  for i in range(4000):
    treated_list, days_taken, final_dose_level, total_inevals, state_log, event_log, end_on_error = simulation_with_debug(sim_type)
    if end_on_error:
      print(event_log)
      break

  Starting_Dose_Level, Highest_Possible_Dose_Level, Lowest_Possible_Dose_Level = orig[0], orig[1], orig[2]
  Waitlist_Time, Course_Length, Screen_Failure_Prob, Inevaluable_Prob = orig[3], orig[4], orig[5], orig[6]

# Simulation Benchmarking (eTable 3 Comparison)

This code block runs simulations and generates a report formatted to match the paper's supplemental **eTable 3**.

### How to customize scenarios
You can add multiple rows to the table by adding dictionaries to the `scenarios_to_run` list. For each scenario, you can define a `name` and any of the following global variables to override:

*   `Starting_Dose_Level` (Default: 2)
*   `Highest_Possible_Dose_Level` (Default: 5)
*   `Lowest_Possible_Dose_Level` (Default: 1)
*   `Waitlist_Time` (Default: 0)
*   `Course_Length` (Default: 28)
*   `Screen_Failure_Prob` (Default: 0.30)
*   `Inevaluable_Prob` (Default: 0.20)

### Function Parameters
1. **`scenarios`**: The list of dictionaries defined above.
2. **`sim_types`**: The simulation IDs to run side-by-side (e.g., `[1, 2]` for 3+3 and IQ 3+3).
3. **`num_repetitions`**: The number of simulated trials per scenario (e.g., 800).


In [20]:
# Define your scenarios here.
# Anything not defined in the dict will use the current global default.
scenarios_to_run = [
    {
        "name": "Scenario A1\nAve Ineval\n20% Ineval",
        "Inevaluable_Prob": 0.20,
        "Starting_Dose_Level": 2
    },
    {
        "name": "Scenario A2\nLow Ineval\n3.6% Ineval",
        "Inevaluable_Prob": 0.036,
        "Starting_Dose_Level": 2
    },
    # {
    #     "name": "Example Scenario ",
    #     "Inevaluable_Prob": 0,
    #     "Screen_Failure_Prob": 0,
    #     "Starting_Dose_Level": 1,
    #     "Highest_Possible_Dose_Level": 1,
    #     "Lowest_Possible_Dose_Level": 1
    # }

]

print("Simulations for 3+3 and IQ 3+3 (each based on 800 simulations)")
run_batch_report(scenarios_to_run, [1, 2], 800)

Simulations for 3+3 and IQ 3+3 (each based on 800 simulations)
+-------------+-------------+-------------+--------------+--------------+-------------+-------------+--------------+------------+------------+-------------+
| Variation   | #Treated    | #Treated    | #Evaluable   | #Evaluable   | #Months     | #Months     | Mean         | 3+3        | IQ 3+3     | DLT rate    |
|             | 3+3         | IQ 3+3      | 3+3          | IQ 3+3       | 3+3         | IQ 3+3      | #DLTs        | % MTD      | % MTD      | (based      |
|             | Mean, Med   | Mean, Med   | Mean, Med    | Mean, Med    | Mean, Med   | Mean, Med   | above        | At level   | At level   | on model)   |
|             | (range)     | (range)     | (range)      | (range)      | (range)     | (range)     | MTD          |            |            |             |
+=============+=============+=============+==============+==============+=============+=============+==============+============+============+==========

In [21]:
# Example block, don't run again
scenarios_to_run = [
    {
        "name": "Example \nScenario 1\n(ineval:0)",
        "Inevaluable_Prob": 0,
        "Screen_Failure_Prob": 0,
        "Starting_Dose_Level": 1,
        "Highest_Possible_Dose_Level": 1,
        "Lowest_Possible_Dose_Level": 1
    },
    {
        "name": "Example \nScenario 2\n(ineval:0.5)",
        "Inevaluable_Prob": 0.5,
        "Screen_Failure_Prob": 0,
        "Starting_Dose_Level": 1,
        "Highest_Possible_Dose_Level": 1,
        "Lowest_Possible_Dose_Level": 1
    }

]

print("Simulations for 3+3 and IQ 3+3 (each based on 800 simulations)")
run_batch_report(scenarios_to_run, [1, 2], 800)

Simulations for 3+3 and IQ 3+3 (each based on 800 simulations)
+--------------+-------------+-------------+--------------+--------------+-------------+-------------+--------------+------------+------------+-------------+
| Variation    | #Treated    | #Treated    | #Evaluable   | #Evaluable   | #Months     | #Months     | Mean         | 3+3        | IQ 3+3     | DLT rate    |
|              | 3+3         | IQ 3+3      | 3+3          | IQ 3+3       | 3+3         | IQ 3+3      | #DLTs        | % MTD      | % MTD      | (based      |
|              | Mean, Med   | Mean, Med   | Mean, Med    | Mean, Med    | Mean, Med   | Mean, Med   | above        | At level   | At level   | on model)   |
|              | (range)     | (range)     | (range)      | (range)      | (range)     | (range)     | MTD          |            |            |             |
+==============+=============+=============+==============+==============+=============+=============+==============+============+============+====

In [22]:
#uncomment this to use as single level dlt. re-run the top version to restore correct values in memory
'''
def get_dlt_probability(CurDoseLevel):

    prob_of_dlt = (
0.15*(CurDoseLevel-2)*(CurDoseLevel-3)*(CurDoseLevel-4)*(CurDoseLevel-5)*(CurDoseLevel-6)/
      ((1-2)*(1-3)*(1-4)*(1-5)*(1-6))
)

    return prob_of_dlt
'''


'\ndef get_dlt_probability(CurDoseLevel):\n\n    prob_of_dlt = (\n0.15*(CurDoseLevel-2)*(CurDoseLevel-3)*(CurDoseLevel-4)*(CurDoseLevel-5)*(CurDoseLevel-6)/\n      ((1-2)*(1-3)*(1-4)*(1-5)*(1-6))\n)\n\n    return prob_of_dlt\n'

In [23]:
# Define your scenarios here.
# Anything not defined in the dict will use the current global default.
scenarios_to_run = [
    {
        "name": "Scenario A1\nAve Ineval\n20% Ineval",
        "Inevaluable_Prob": 0.20,
        "Starting_Dose_Level": 2
    },
    {
        "name": "Scenario A2\nLow Ineval\n3.6% Ineval",
        "Inevaluable_Prob": 0.036,
        "Starting_Dose_Level": 2
    }
]

print("Simulations for Rolling 6 and IQ Rolling 6 (each based on 800 simulations)")
run_batch_report(scenarios_to_run, [3, 4], 800)

Simulations for Rolling 6 and IQ Rolling 6 (each based on 800 simulations)
+-------------+-------------+----------------+--------------+----------------+-------------+----------------+--------------------+-------------+----------------+-------------+
| Variation   | #Treated    | #Treated       | #Evaluable   | #Evaluable     | #Months     | #Months        | Mean               | Rolling 6   | IQ Rolling 6   | DLT rate    |
|             | Rolling 6   | IQ Rolling 6   | Rolling 6    | IQ Rolling 6   | Rolling 6   | IQ Rolling 6   | #DLTs              | % MTD       | % MTD          | (based      |
|             | Mean, Med   | Mean, Med      | Mean, Med    | Mean, Med      | Mean, Med   | Mean, Med      | above              | At level    | At level       | on model)   |
|             | (range)     | (range)        | (range)      | (range)        | (range)     | (range)        | MTD                |             |                |             |
+=============+=============+================

# Individual Simulation Event Log

Use this block to run a **single trial** and view the detailed step-by-step event logs. This is useful for debugging patient flow, dose escalations, and timing logic.

### How to use:
Call `run_debug_test()` and type the values you want to change. If you don't type a variable, it uses the default.

### Function Parameters
*   `sim_type`: 1=3+3, 2=IQ 3+3, 3=Rolling 6, 4=IQ Rolling 6
*   `sdl`: Starting Dose Level (Default: 2)
*   `hpd`: Highest Possible Dose (Default: 5)
*   `lpd`: Lowest Possible Dose (Default: 1)
*   `wt`: Waitlist Time (Default: 0)
*   `cl`: Course Length (Default: 28)
*   `sfp`: Screen Failure Prob (Default: 0.30)
*   `ip`: Inevaluable Prob (Default: 0.20)




In [24]:
run_debug_test(sim_type=1, sdl=2, ip=0.2)
